In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('layer_9_train.csv')
valid = pd.read_csv('layer_9_valid.csv')
test = pd.read_csv('layer_9_test.csv')

In [3]:
train_X = train[train['label_2'].notna()]                                              # NaN values are dropped
train_X.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
train_y = train[train['label_2'].notna()]['label_2']

valid_X = valid[valid['label_2'].notna()]                                             # NaN values are dropped
valid_X.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
valid_y = valid[valid['label_2'].notna()]['label_2']

test_X = test[test['label_2'].notna()]                                             # NaN values are dropped
test_X.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
test_y = test[test['label_2'].notna()]['label_2']

C:\Users\satha\AppData\Local\Temp\ipykernel_5420\490337159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
C:\Users\satha\AppData\Local\Temp\ipykernel_5420\490337159.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_X.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
C:\Users\satha\AppData\Local\Temp\ipykernel_5420\490337159.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_X = scaler.fit_transform(train_X)
valid_X = scaler.fit_transform(valid_X)
test_X = scaler.fit_transform(test_X)

In [5]:
from sklearn.decomposition import PCA

pca = PCA(
    n_components = 0.994,
    svd_solver='full',
    whiten=True
    )
pca.fit(train_X)

train_X_new = pca.transform(train_X)
valid_X_new = pca.transform(valid_X)
test_X_new = pca.transform(test_X)

In [6]:
train_X_new.shape

(28040, 621)

In [7]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

base_svm = SVC(
    kernel='linear',
    C=1,
    gamma='auto',
)

bagged_svm = BaggingClassifier(
    base_svm, 
    n_estimators=5, 
    random_state=10
)

bagged_svm.fit(train_X_new, train_y)

pred_y = bagged_svm.predict(valid_X_new)

accuracy_score_valid = accuracy_score(valid_y, pred_y)
accuracy_score_valid

0.7730978260869565

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# number of splits for k-fold cross-validation
num_splits = 5

kf = KFold(
    n_splits=num_splits, 
    shuffle=True, 
    random_state=42
)

cross_val_scores = cross_val_score(bagged_svm, train_X_new, train_y, cv=kf, scoring='accuracy')

print("Cross-Validation Scores :", cross_val_scores)
print("Mean Accuracy =",  np.mean(cross_val_scores))
print("Standard Deviation =",  np.std(cross_val_scores))

In [8]:
pred_test_y = bagged_svm.predict(test_X_new)

accuracy_score_valid = accuracy_score(test_y, pred_test_y)
accuracy_score_valid

0.7997311827956989

In [9]:
rows, columns = test_X_new.shape

new_columns = {
    'label_2' : pred_test_y
}

data_frame = pd.DataFrame(new_columns)

csv_file_path = 'test_label_2_final.csv'
data_frame.to_csv(csv_file_path, index=False)